# Imports

In [ ]:
# import pandas as pd
# import warnings
# warnings.filterwarnings("ignore")
# from glob import glob
# from tqdm.notebook import tqdm
# import matplotlib
# matplotlib.rcParams.update({'font.size': 22})
# from sklearn.metrics import accuracy_score
# from tensorflow.keras import layers
# from tensorflow.keras.applications import ResNet50, DenseNet121, Xception
# from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras import models
# from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
# import tensorflow.keras.backend as K
# from tensorflow.math import confusion_matrix

### this work is inspired by:

https://www.kaggle.com/rbhambri/covid-detection-studies-eda-viz

https://www.kaggle.com/rbhambri/chexnet-transfer-learning-binary-image-clf
    
https://www.kaggle.com/sinamhd9/classification-model

https://www.kaggle.com/rbhambri/chest-x-ray-abnormalities-bams-keras-pipeline

https://www.kaggle.com/rbhambri/densenet-weights-nih-coursera-ai4m

https://www.coursera.org/specializations/ai-for-medicine

https://www.kaggle.com/rbhambri/chest-x-ray-abnormalities-densenet-pipeline

https://www.kaggle.com/c/siim-covid19-detection/discussion/242606

https://www.kaggle.com/saeedniksaz/transferlearning-with-vgg16



# Data manipulations

In [ ]:
import pandas as pd


In [ ]:
image_df = pd.read_csv('../input/siim-covid19-detection/train_image_level.csv')
display(image_df.head(3))
print(image_df.shape)

In [ ]:
study_df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
display(study_df.head(3))
print(study_df.shape)

In [ ]:
# df_sampleSub = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
# display(df_sampleSub.head(3))
# print(df_sampleSub.shape)

In [ ]:
study_df['id'] = study_df['id'].str.replace('_study',"")
study_df['StudyInstanceUID'] = study_df['id']
study_df.head(3)

### for simplicity keeping studies with only 1 img per study - right now

In [ ]:
def get_absolute_file_paths(directory):
    all_abs_file_paths = []
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            all_abs_file_paths.append(os.path.abspath(os.path.join(dirpath, f)))
    return all_abs_file_paths

In [ ]:
from tqdm.notebook import tqdm; tqdm.pandas();


In [ ]:
import os


In [ ]:
study_df["study_dir"] = "/kaggle/input/siim-covid19-detection/train/"+study_df["id"]
study_df["images_per_study"] = study_df.study_dir.progress_apply(lambda x: len(get_absolute_file_paths(x)))

In [ ]:
study_df.head()

In [ ]:
study_df.images_per_study.value_counts()

In [ ]:
unique_studies = study_df[study_df.images_per_study == 1]

In [ ]:
unique_studies.head()

In [ ]:
unique_studies.images_per_study.value_counts()

In [ ]:
unique_studies_ids = list(unique_studies.id)

In [ ]:
unique_studies_ids[:10]

In [ ]:
len(unique_studies_ids)

### only keep images for these studies

In [ ]:
image_df.head()

In [ ]:
image_df_unique = image_df[image_df.StudyInstanceUID.isin(unique_studies_ids)]

In [ ]:
image_df_unique.shape

In [ ]:
image_df.shape

### now merge

In [ ]:
df_train = image_df_unique.merge(study_df, 
                                 on='StudyInstanceUID',
                                 suffixes=('_image', '_study'))
df_train.head(3)

In [ ]:
df_train.columns

In [ ]:
# train_dir_jpg = '../input/covid-jpg-512/train'
# train_dir_origin ='../input/siim-covid19-detection/train'
# paths_original = []
# paths_jpg = []
# for _, row in tqdm(df_train.iterrows()):
#     image_id = row['id'].split('_')[0]
#     study_id = row['StudyInstanceUID']
#     image_path_jpg = glob(f'{train_dir_jpg}/{image_id}.jpg')
#     image_path_original = glob(f'{train_dir_origin}/{study_id}/*/{image_id}.dcm')
#     paths_jpg.append(image_path_jpg)
#     paths_original.append(image_path_original)

In [ ]:
# df_train['path'] = paths_jpg
# df_train['origin'] = paths_original
# df_train.head(3)

In [ ]:
df_train.loc[df_train['Negative for Pneumonia']==1, 'study_label'] = 'negative'
df_train.loc[df_train['Typical Appearance']==1, 'study_label'] = 'typical'
df_train.loc[df_train['Indeterminate Appearance']==1, 'study_label'] = 'indeterminate'
df_train.loc[df_train['Atypical Appearance']==1, 'study_label'] = 'atypical'
df_train.drop(['Negative for Pneumonia','Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], axis=1, inplace=True)


In [ ]:
df_train['id_image'] = df_train['id_image'].str.replace('_image', '.jpg')
df_train['image_label'] = df_train['label'].str.split().apply(lambda x : x[0])
df_train.head(3)

In [ ]:
df_size = pd.read_csv('../input/covid-jpg-512/size.csv')
df_size['id_image'] = df_size['id']
df_size.head(3)

In [ ]:
df_train = df_train.merge(df_size, on='id_image')
df_train.head(3)

In [ ]:
df_train.shape

In [ ]:
df_train.study_label.value_counts()

In [ ]:
df_train.image_label.value_counts()

In [ ]:
# df_train.id_image.value_counts()

In [ ]:
# df_train.id_study.value_counts()

In [ ]:
# df_train.describe()

### data is imbalanced.

oversample the minority class.

In [ ]:
df_train_atypical = df_train[df_train.study_label == 'atypical']

In [ ]:
df_train_atypical.shape

In [ ]:
df_train_augmented_list = [df_train, df_train_atypical]

In [ ]:
df_train_augmented = pd.concat(df_train_augmented_list, ignore_index=True)

In [ ]:
df_train_augmented.shape

In [ ]:
df_train_augmented.study_label.value_counts()

In [ ]:
df_train_augmented.image_label.value_counts()

### now use this df for further processing

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
from ast import literal_eval


In [ ]:
from matplotlib.patches import Rectangle


In [ ]:
df_train['id'][0]

In [ ]:
# ! ls ../input/siimcovid19-1024-jpg-image-dataset/train

In [ ]:
n = 20
train_dir = '../input/covid-jpg-512/train'
# train_dir = '../input/siimcovid19-1024-jpg-image-dataset/train'


fig, axs = plt.subplots(4, 5, figsize=(20,20))
fig.subplots_adjust(hspace=.2, wspace=.2)
axs = axs.ravel()
for i in range(n):
#     print('----------')
    image_id = df_train['id'][i]
#     print('ix, image_id=', i, image_id)

    img = cv2.imread(os.path.join(train_dir, image_id))
    axs[i].imshow(img)
    
    study_label = df_train['study_label'][i]
    image_label = df_train['image_label'][i]
#     print('study_label=', study_label)
#     print('image_label=', image_label)
    
    if type(df_train['boxes'][i])==str:
#         print('box seen for i=', i)
        boxes = literal_eval(df_train['boxes'][i])
        
        for box in boxes:
#             print('box=', box)
            axs[i].add_patch(Rectangle((box['x']*(512/df_train['dim1'][i]), box['y']*(512/df_train['dim0'][i])), box['width']*(512/df_train['dim1'][i]), box['height']*(512/df_train['dim0'][i]), fill=0, color='y', linewidth=2))
        axs[i].set_title(str(i) + ',' + study_label + ',' + image_label)
            
    else:
#         print('box NOT seen for i=', i)
        axs[i].set_title(str(i) + ',' + study_label + ',' + image_label)


# PreProcessing

In [ ]:
from skimage import exposure
import numpy as np


In [ ]:
# def preprocess_image(img):
#     equ_img = exposure.equalize_hist(img)
#     return equ_img

# im= cv2.imread('../input/covid-jpg-512/train/007cf31356c6.jpg')
# im2 = preprocess_image(im)
# res = np.concatenate((im/255, im2), axis=1)
# plt.imshow(res)
# plt.show()

In [ ]:
img.shape

# ImageGenerators and Augmentations

In [ ]:
# img_size = 1024
img_size = 512
# img_size = 299


In [ ]:
# batch_size = 32
batch_size = 16


In [ ]:
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
image_generator = ImageDataGenerator(
#         rescale = 1./255,
        validation_split=0.25,
        rotation_range=5,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        brightness_range = [0.8, 1.1],
)

image_generator_valid = ImageDataGenerator(validation_split=0.25,
#                                            rescale = 1./255,  
                                          )


In [ ]:
df_train = df_train_augmented

In [ ]:
df_train.head()

In [ ]:
train_dir

In [ ]:
train_generator = image_generator.flow_from_dataframe(
        dataframe = df_train,
        directory = train_dir,
        x_col = 'id',
        y_col =  'study_label',  
        target_size=(img_size, img_size),
        batch_size=batch_size,
#         class_mode='binary',
        subset='training', 
        seed = 23) 

valid_generator = image_generator_valid.flow_from_dataframe(
    dataframe = df_train,
    directory = train_dir,
    x_col = 'id',
    y_col = 'study_label',
    target_size=(img_size, img_size),
    batch_size=batch_size,
#     class_mode='binary',
    subset='validation', 
    shuffle=False, 
    seed=23) 


In [ ]:
# for j in range(2):
#     aug_images = [train_generator[0][0][j] for i in range(5)]
#     fig, axes = plt.subplots(1, 5, figsize=(24,24))
#     axes = axes.flatten()
#     for img, ax in zip(aug_images, axes):
#         ax.imshow(img)
#         ax.axis('off')
# plt.tight_layout()
# plt.show()

In [ ]:
# for j in range(2):
#     aug_images = [valid_generator[0][0][j] for i in range(5)]
#     fig, axes = plt.subplots(1, 5, figsize=(24,24))
#     axes = axes.flatten()
#     for img, ax in zip(aug_images, axes):
#         ax.imshow(img)
#         ax.axis('off')
# plt.tight_layout()
# plt.show()

# Architecture

In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model



In [ ]:
from tensorflow.keras.models import Model, Sequential


In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D,  BatchNormalization, Activation
# from tensorflow.keras import models


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping


In [ ]:
img_size

In [ ]:
from tensorflow.keras.applications.vgg16 import  VGG16
from tensorflow.keras.applications import DenseNet121


In [ ]:
def build_chextnet_model_v1():
    """
    v1 - uses densenet + chextnet weights
    """
    # load model design
    pre_model = DenseNet121(weights=None,
                        include_top=False,
                        input_shape=(img_size,img_size,3)
                        )
    out = Dense(14, activation='sigmoid')(pre_model.output)
    pre_model = Model(inputs=pre_model.input, outputs=out) 
    
    # load model wieghts
    chex_weights_path = '../input/chexnet-weights/brucechou1983_CheXNet_Keras_0.3.0_weights.h5'
    pre_model.load_weights(chex_weights_path)

    # make layers trainable?
    # pre_model.trainable = False
    pre_model.trainable = True

    # get summary/print
    pre_model.summary()
    
    
    # add future layers.
    # last_layer = pre_model.get_layer('conv5_block16_concat')
    last_layer = pre_model.layers[-2]

    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output
#     last_layer

    # Flatten the output layer to 1 dimension
    # x = Flatten()(last_output)
    x = GlobalAveragePooling2D()(last_output)

    # Add a fully connected layer with 512 hidden units and ReLU activation
    # x = Dense(512, activation='relu')(x)
    # Add a dropout rate of 0.2
    # x = Dropout(0.2)(x)                  


    # # Add a fully connected layer with 128 hidden units and ReLU activation
    # x = Dense(128, activation='relu')(x)


    # Add final classification layer
    x = Dense(4, activation='softmax')(x)

    # final model
    model = Model( pre_model.input, x) 

    # model.summary()
    # plot_model(model)

    return model


In [ ]:
def build_vanilla_cnn_model_v1():
    in1 = tf.keras.layers.Input(shape=(img_size, img_size, 3))
    
#     out1 = tf.keras.layers.Conv2D(4,(3,3),activation="relu")(in1)
    out1 = tf.keras.layers.Conv2D(32,(3,3),
                                  activation="relu",
                                  padding='same')(in1)
    out1 = tf.keras.layers.MaxPooling2D((2,2))(out1)
    
    out1 = tf.keras.layers.Conv2D(32,(3,3),
                                  activation="relu",
                                  padding='same')(out1)
    out1 = tf.keras.layers.MaxPooling2D((2,2))(out1)

    out1 = tf.keras.layers.Flatten()(out1)
    
    out2 = tf.keras.layers.Dense(30,activation="relu")(out1)
    out2 = tf.keras.layers.Dense(30,activation="relu")(out2)
    
    
    out2 = Dense(4, 
                 activation='softmax',
                 name='class_out',
                 kernel_regularizer=regularizers.l2(0.01))(out2)


    model = tf.keras.Model(inputs=in1,
                           outputs=out2)

    model.summary()
    return model


In [ ]:
def build_densenet_coursera_model_v1():
    """
    v1 - uses densenet + coursera model
    https://www.kaggle.com/rbhambri/densenet-weights-nih-coursera-ai4m
    """
    # load model design
    # also load model wieghts
    nih_weights_path = '../input/densenet-weights-nih-coursera-ai4m/densenet.hdf5'
    base_model = DenseNet121(weights=nih_weights_path, 
                             input_shape=(img_size,img_size,3),
                             include_top=False)

    # get summary/print
#     base_model.summary()

    last_output = base_model.output
#     last_layer

    # Flatten the output layer to 1 dimension
    # x = Flatten()(last_output)
    x = GlobalAveragePooling2D()(last_output)

    # Add final classification layer
    x = Dense(4, activation='softmax')(x)

    # final model
    model = Model(base_model.input, x) 

    model.summary()
    plot_model(model)

    return model


In [ ]:
def build_vgg16_model_v1():
    """
    v1 - https://www.kaggle.com/saeedniksaz/transferlearning-with-vgg16
    """
    base_model = VGG16(input_shape=(img_size,img_size,3), 
                         include_top=False,
                         weights="imagenet")
    # get summary/print
#     base_model.summary()

    for layer in base_model.layers:
        layer.trainable = False
        
    model = Sequential()
    model.add(base_model)
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(BatchNormalization())
    
    model.add(Dense(256,kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(32,kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Dense(4,activation='softmax'))
        
    model.summary()
    plot_model(model)

    return model


In [ ]:
# model = build_vanilla_cnn_model_v1()
model = build_chextnet_model_v1()
# model = build_densenet_coursera_model_v1()
# model = build_vgg16_model_v1()


In [ ]:
# metrics = ['categorical_accuracy', 'accuracy']
# metrics = [tf.keras.metrics.AUC(), 'accuracy']
metrics = [ 'accuracy', tf.keras.metrics.AUC()]



In [ ]:
model.compile(Adam(lr=1e-3),
              loss='categorical_crossentropy',
              metrics=metrics)

In [ ]:
model

In [ ]:
# rlr = ReduceLROnPlateau(monitor = 'val_accuracy', 
#                         factor = 0.2, 
#                         patience = 2, 
#                         verbose = 1, 
#                         min_delta = 1e-4, 
#                         min_lr = 1e-4, 
#                         mode = 'max')

rlr = ReduceLROnPlateau(monitor = 'val_loss', 
                        factor = 0.1, 
                        patience = 2, 
                        verbose = 1, 
                        min_delta = 1e-4, 
                        min_lr = 1e-6, 
                        mode = 'min')


In [ ]:
# es = EarlyStopping(monitor = 'val_accuracy', 
#                    min_delta = 1e-4, 
#                    patience = 3, 
#                    mode = 'max', 
#                    restore_best_weights = True, 
#                    verbose = 1)

es = EarlyStopping(monitor = 'val_loss', 
                   min_delta = 1e-4, 
                   patience = 3, 
                   mode = 'min', 
                   restore_best_weights = True, 
                   verbose = 1)


In [ ]:
# model_name = 'vgg16_model_512_july11.h5'
model_name = 'chextnet_model_512_july11.h5'


In [ ]:
# ckp = ModelCheckpoint('model.h5',
#                       monitor = 'val_accuracy',
#                       verbose = 0, 
#                       save_best_only = True, 
#                       mode = 'max')


ckp = ModelCheckpoint(model_name,
                      monitor = 'val_loss',
                      verbose = 0, 
                      save_best_only = True, 
                      mode = 'min')


In [ ]:
# class_weight = {0: 2,1: 1, 2: 1,3: 1}

In [ ]:
epochs = 10
# epochs = 5
# epochs = 2


In [ ]:
history = model.fit(
      train_generator,
      validation_data=valid_generator,
      epochs=epochs,
      callbacks=[es, ckp, rlr],
#     class_weight=class_weight
)

In [ ]:
# abc

In [ ]:
# ! ls ../input/image-clf-chexnet-vanilla-cnn
! ls

In [ ]:
# model.load_weights('../input/image-clf-chexnet-vanilla-cnn/model.h5')
# model.load_weights('./chextnet_model_512_july11.h5')


### model perf

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
# from tensorflow.math import confusion_matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
actual =  valid_generator.labels
preds = np.argmax(model.predict(valid_generator), axis=1)
cfmx = confusion_matrix(actual, preds)
acc = accuracy_score(actual, preds)

In [ ]:
model.evaluate(valid_generator)

In [ ]:
actual[:25]

In [ ]:
preds[:25]

In [ ]:
cfmx

In [ ]:
acc

In [ ]:
print(classification_report(actual, preds))


In [ ]:
df_train.study_label.value_counts()

In [ ]:
valid_generator.class_indices

In [ ]:
c = model.predict(valid_generator)

In [ ]:
c.shape

In [ ]:
# c[0]

In [ ]:
ix = 3

In [ ]:
actual[ix]

In [ ]:
op = c[ix]

In [ ]:
op

In [ ]:
for ix, y_pred in enumerate(list(op)):
    if y_pred > 0.5:
#         print(ix, y_pred, classes[ix])        
        print(ix, y_pred)

### address class imbalance

now try focal loss? https://towardsdatascience.com/a-loss-function-suitable-for-class-imbalanced-data-focal-loss-af1702d75d75

OR specify in model.fit

https://stackoverflow.com/questions/44716150/how-can-i-assign-a-class-weight-in-keras-in-a-simple-way/44721883

https://github.com/keras-team/keras/issues/1875


### TODO

check old kaggle competition on chest xray - done - doing better than this

add metrics - https://keras.io/api/metrics/classification_metrics/ - done.

add weighted loss function / class weights in model/ balance train df - done with oversampling

check coursera assignments

try better models.

gradcam?
